In [19]:
# 파이썬에 없는 라이브러리 설치
!pip install feedparser
!pip install newspaper3k
!pip install konlpy

In [20]:
import feedparser   # URL을 입력받으면 페이지에 있는 <item> </item>을 가져오는 것
from newspaper import Article  # 기사를 가져오기 위해 사용
from konlpy.tag import Okt     # 한국어 사용하기 위한 라이브러리
from collections import Counter # 반복 가능한 객체(예: 리스트 또는 문자열)의 요소들의 개수를 셀 때 유용하게 사용
from bs4 import BeautifulSoup  # 웹 스크레이핑(scraping) 작업을 수행하여 웹 페이지에서 원하는 정보를 추출

In [21]:
# [단계1] 모든 RSS파일을 돌아다니며 기사의 제목/link를 추출함
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml"]
# 아래 함수는 RSS목록의 list안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(urls):
    array_rss = [] # 함수 시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사를 채울 것임
    titles_rss = set() # [중복기사제거] 기사제목들의 집합을 구성(집합은 중복을 불허하기 때문에)
    for url in urls: # 주어진 4개의 RSS파일을 하나씩 방문(총 4번 돌것이고, 901, 902, 903, 904)
        print("[Crawl URL]", url) # 현재 어디에 있는 출력
        parse_rss = feedparser.parse(url) # 현재 url을 파싱한 후에 결과를 parse_rss에 저장
        for p in parse_rss.entries : # parse_rss에 있는 모든 entrires/기사를 검색
            if p.title not in titles_rss: # [중복기사제거] 만약에 titles_rss 라는 집합에 방금 찾은 제목이 없으면 추가
                array_rss.append({'title' : p.title, 'link':p.link}) # 기사에서 제목/link 추출후 리스트에 추가
                titles_rss.add(p.title) # [중복기사제거] 집합에 현재 기사제목이 없을 때만 추가
            else:
                print("Duplicated Article:", p.title) # [중복기사제거] 중복된다고 판단된 기사 제목 출력
    return array_rss

list_articles = crawl_rss(urls) # 실제 crawl_rss 실행
print(list_articles)

[Crawl URL] http://rss.etnews.com/Section901.xml
[Crawl URL] http://rss.etnews.com/Section902.xml
Duplicated Article: '디지털 전환' 증권가, 전산운용비 늘리고 지점 줄이고
Duplicated Article: [이슈플러스]윤 대통령 “의료개혁 계속 추진”…의정갈등 장기전
[Crawl URL] http://rss.etnews.com/Section903.xml
Duplicated Article: 기아, 3000만원대 전기차 내년 유럽 출시
Duplicated Article: 삼성, 美 추가투자...파운드리·패키징 다 갖춘다
Duplicated Article: 美, 삼성 반도체 보조금 64억달러 확정
Duplicated Article: 현대차 아이오닉 6 N, 獨 뉘르부르크링 '극한 주행' 테스트 돌입
Duplicated Article: 정부, 통합보안 모델 만든다…'K-시큐리티 얼라이언스' 속도
[Crawl URL] http://rss.etnews.com/Section904.xml
Duplicated Article: '디지털 전환' 증권가, 전산운용비 늘리고 지점 줄이고
Duplicated Article: [이슈플러스]윤 대통령 “의료개혁 계속 추진”…의정갈등 장기전
Duplicated Article: [르포] 예술작품이 된 'LG 시그니처'…유럽 상위 1% 정조준
Duplicated Article: 삼성전자, AI시대 디자인 철학 '본질·혁신·조화' 선언
Duplicated Article: IAEA “이스라엘 '이란 핵시설' 공격 가능성에 우려”
Duplicated Article: 테슬라, 판매부진에 전 세계 직원 10% 이상 감축… 주가도 급락
Duplicated Article: 삼성 갤S22·갤Z4, 내달 3일부터 AI 기능 탑재
Duplicated Article: 日 외교청서 “독도는 일본 땅…징용 판결 수용 불가”
Duplicated Article: 전남대, 세계

In [22]:
# [단계 #2] list에 조잰하는 모든 링크를 돌아다니면서 본문 test를 긁어오기
    # 아래의 함수는 하나의 url을 입력받아서, 링크를 타고 들어가, 그 안에서 title과 test를 추출
    # 언어는 한글로 기본 지정

def crawl_article(url, language='ko'): # url을 타고들어가서 제목과 기사를 뽑는 코드
    print("[Crawl Article]", url) # 현재 title과 test를 추출한 url을 프린트해줌
    a = Article(url, language = language) # Article을 사용하여 그 URL을 입력하고, 언어옵션지정(한글어기본), 결과를 a에 저장
    a.download()
    a.parse()
    return a.title, a.text # a에 해당하는 title과 본문(text)을 출력한다.

def preprocessing(text):
    test_article = BeautifulSoup(test, 'html5lib').get_test() # html 태그 제거
    return test_article

for article in list_articles : # list에 있는 모든 기사를 하나씩 방문
    _, test = crawl_article(article['link']) # 그 기사의 link를 crawl_article 함수에 넣어 본문(test)추출
    article['text'] = test # 추출된 본문을 'test'라는 속성으로 새로 만들어서 저장
print(list_articles[0]) # 첫번째 기사를 출력하여 title, link, text가 모두 있는 확인

[Crawl Article] https://www.etnews.com/20240416000158
[Crawl Article] https://www.etnews.com/20240416000155
[Crawl Article] https://www.etnews.com/20240416000129
[Crawl Article] https://www.etnews.com/20240416000113
[Crawl Article] https://www.etnews.com/20240416000109
[Crawl Article] https://www.etnews.com/20240416000096
[Crawl Article] https://www.etnews.com/20240416000037
[Crawl Article] https://www.etnews.com/20240416000026
[Crawl Article] https://www.etnews.com/20240416000016
[Crawl Article] https://www.etnews.com/20240416000004
[Crawl Article] https://www.etnews.com/20240416000003
[Crawl Article] https://www.etnews.com/20240415000464
[Crawl Article] https://www.etnews.com/20240415000426
[Crawl Article] https://www.etnews.com/20240415000349
[Crawl Article] https://www.etnews.com/20240415000436
[Crawl Article] https://www.etnews.com/20240415000433
[Crawl Article] https://www.etnews.com/20240412000017
[Crawl Article] https://www.etnews.com/20240415000209
[Crawl Article] https://www.

In [23]:
# [단계 #3] 모든 본문 text를 돌아다니면서 명서(키워드, 빈도수)를 추출
def get_keywords(text, num_keywords=10): # 키워드추출함수, 디폴트로 10개 추출한다.
    spliter = Okt() # konlpy 에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
    nouns = spliter.nouns(text) # spliter에서 nouns함수를 불러 text를 넣으면 그 text의 명사만 출력
    count = Counter(nouns) # 추출된 명사들의 출현 빈도 추출
    list_keywords = [] # 비어있는 키워드 리스트를 먼저 만든 후에 .....
    for n, c in count.most_common(num_keywords): # 가장 출현빈도 높은 명사부터 순차적으로 10번 출력
        item = {'keyword':n, 'count':c} # 리스트에 저장은, {'keyword':n, 'count':c} 형식으로 저장됨
        list_keywords.append(item) # 이런 저장된 포맷으로 이를 list_keywords에 붙여 나감
    return list_keywords

for article in list_articles:  # 모든 기사를 돌아다니면서 text에서 명사추출/키워드빈도추출
    keywords = get_keywords(article['text']) # get_keywords 함수로 주어진 text의 명사/키워드 추출
    article['keywords'] = keywords # 추출된 키워드와 빈도를 list_articles의 'keywords'로 저장
print(list_articles[0]) # 첫번째 기사를 출력하여 title, link, text, keywords가 모두 있는지 확

{'title': "'디지털 전환' 증권가, 전산운용비 늘리고 지점 줄이고", 'link': 'https://www.etnews.com/20240416000158', 'text': "금융권에 디지털 DNA가 빠르게 이식되고 있는 가운데 증권가에서도 IT 역량 강화를 위한 투자를 늘렸다. 오프라인 지점 수를 줄이는 한편, 모바일트레이딩시스템(MTS)을 고도화하는 등 디지털 고객 대상 전략을 확대편성한 모습이다.\n\n\n\n금융정보통계시스템에 따르면 지난해 국내 증권사 48곳의 전산운용비는 전년 대비 9.35% 늘었다. 3년 전과 비교했을 때는 51.26%로 증가 폭이 더욱 뚜렷하다.\n\n\n\n주요 7대 증권사 중에서는 KB증권이 22.9%로 가장 많이 전산운용비를 늘렸다. 이어 신한투자증권 22.43%, 메리츠증권 10.18%, 삼성증권 9.13%, 한국투자증권 5.5%, 미래에셋증권 4.55% 순으로 집계됐다. NH투자증권은 2.8% 감소했다. 특히 KB증권과 신한투자증권은 3년 새 전산 운영비 규모를 각 205.7%, 160.77% 대폭 확장했다.\n\n\n\n모바일 트레이딩 시스템(MTS)에 인공지능(AI) 등 신기술을 지속 탑재하는 등 디지털 서비스 고도화에 따른 결과로 분석된다. 대형 증권사를 비롯, 중소형 증권사도 MTS 업데이트를 실시하면서 '엄지족' 이용자 확보에 만전을 기하고 있다.\n\n\n\nKB증권은 MTS에 고객 질문에 실시간 응답해 주는 금융 비서 '스톡 AI'를 선보였다. 한국투자증권은 로보어드바이저 랩(WRAP) 상품 추천해 주는 '마이 AI'를 출시했다. IBK투자증권은 올해 10월 공개를 목표로 MTS 고도화 작업을 진행 중이다. 현대차증권은 신규 MTS '내일'을 선보인 바 있다.\n\n\n\n토큰증권(ST) 등 신시장에 대응하기 위한 전산 개발도 비용 증가를 이끌었다. 하나증권은 MTS에서 ST를 거래할 수 있는 시스템을 개발 중이다. 다수 증권사도 시장 선점을 위해 상호 업무협약을 체결하고 인프라 구축에 박차를 가하고 

In [24]:
# [단계 #3] 본문 text를 코사인유사도를 활용하여 판단 (예: 첫번째 기사와 두번째 기사의 유사도 판단)
# 코사인 유사도 함수 구현

# 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer       # 벡터화
from sklearn.metrics.pairwise import cosine_similarity            # 코사인 유사도를 판단하기 위함

# 리스트에 있는 모든 기사들을 돌아다니면서 text를 다 추출함
text_articles = []
for article in list_articles:
  text_articles.append(article['text'])

# [위의 세줄과 동일]
# text_articles = [article['text'] for article in list_articles]

# 주어진 문장을 벡터로 만드는 객체 생성
vector = TfidfVectorizer(max_features=5000)

# fit_transform함수를 이용하여 5000사이즈 벡터 생성
tfidf_vectors = vector.fit_transform(text_articles)
# tfidf_vector는 모든 기사를 5000사이즈 벡터로 만든 후에 저장한 변수

# 2024 04 16. 11:56 기준 총 73개 기사가 있음
print(tfidf_vectors.shape)
# 첫번째 기사와 두번재 기사 비교
print(cosine_similarity(tfidf_vectors[0], tfidf_vectors[1]))

(73, 5000)
[[0.03172122]]


In [25]:
#[단계 #4] 글( 단어, 문장 )을 입력받아 코사인유사도로 가장 유사한 문서 1개 검출
query = [input()] # 하나의 질의를 입력받음
target = vector.transform(query) # 입력받은 질의를 5000사이즈 벡터로 변환(벡터이름 : target)
similarities = [(i, cosine_similarity(target, sim)) for i, sim in enumerate(tfidf_vectors)]
# (i, 코사인유사도)로 튜플을 만들어줌. i는 그냥 증가값, 코사인유사도는 target과 전체 문서들간의 유사도
index, max_similarity = max(similarities, key = lambda x:x[1])
# 지금 현재 (i, 유사도)로 값이 리스트로 들어옴. 키 값을 앞에 있는 i인지 뒤에있는 유사도인지 정해야함
# key의 세팅을 보니. lambda로 하여, x:x[1]로 지정한 것은
# x를 키로하되, x[1]에 해당하는 유사도 값이 기준임(즉, x[0]이었다면 i, x[1]이었다면 유사도임)
# 그리고 최대값을 구하라는 max함수를 사용하였기에 x[1]에 해당하는 유사도가 제일 큰, 인덱스 index와 유사도를 뽑음
# 따라서 출력을 index에는 유사도가 제일 좋을 때의 i값이 들어가고, max_similarity에는 최대 유사값이 들어감
print('-----'*10)
print('유사도가 가장 높은 기사의 인덱스:', index)
print('유사도 : %5.2f%%' %(100*max_similarity.item()))
print('유사도가 가장 높은 기사의 제목 : ', list_articles[index]['title'])
print('유사도가 가장 높은 기사의 URL : ', list_articles[index]['link'])

삼성
--------------------------------------------------
유사도가 가장 높은 기사의 인덱스: 14
유사도 : 14.19%
유사도가 가장 높은 기사의 제목 :  美, 삼성 반도체 보조금 64억달러 확정
유사도가 가장 높은 기사의 URL :  https://www.etnews.com/20240415000436


In [26]:
# [단계 #4] 글(단어, 문장)을 입력받아 코사인 유사도로 가장 유사한 문서 1개 검출
query = [input()]          # 하나의 질의를 입력받음
target = vector.transform(query)      # 입력받은 질의를 5000사이즈 벡터로 변환 (벡터이름: target)
similarites = [(i, cosine_similarity(target, sim)) for i, sim in enumerate(tfidf_vectors) ]
# (i, 코사인 유사도)로 튜플을 만들어줌. i는 그냥 증가 값. 코사인 유사도는 target과 전체 문서들 간의 유사도

index, max_similarity = max(similarites, key = lambda x:x[1])
# 지금 현재 (i, 유사도)로 값이 리스트로 들어옴. 키 값이 앞에 있는 i인지 뒤에있는 유사도인지 정해야 함
# key의 세팅을 보니, lambda로 하여, x:x[1]로 지정한 것은
# x를 키로 하되, x[1]에 해당하는 유사도값이 기준임(즉, x[0] 이었다면 i, x[1]이었다면 유사도임)
# 그리고 최대값을 구하라는 max함수를 사용하였기에 x[1]에 해당하는 유사도가 제일 큰, 인덱스 index와 유사도를 뽑음
# 따라서 출력을 index에는 유사도가 제일 좋을 때의 i값이 들어가고, max_similarity에는 최대 유사값이 들어감

print("="*30)
print("유사도 가장 높은 기사의 인덱스", index)
print("유사도: %5.2f%%" %(100*max_similarity.item()))
print("유사도가 가장 높은 기사의 제목 :", list_articles[index]['title'])
print("유사도가 가장 높은 기사의 URL : ", list_articles[index]['link'])

삼성
유사도 가장 높은 기사의 인덱스 14
유사도: 14.19%
유사도가 가장 높은 기사의 제목 : 美, 삼성 반도체 보조금 64억달러 확정
유사도가 가장 높은 기사의 URL :  https://www.etnews.com/20240415000436


In [39]:
# [단계 4] 질의를 입력받아서 그 질의어를 가지고 있는 기사들을 출력
query = input("질의어 입력: ")
def search_articles(query, list_keywords): # 질의어가 키워드리스트에 있으면 빈도수를 출력(없으면 0)
    num_words = 0 # 질의어가 해당 문서에서 몇 번 나오는지 출력하기 위한 변수의 초기값은 0으로 설정
    for kw in list_keywords : # 키워드 리스트를 검색합니다.
        if query == kw['keyword']: # 만약에 질의어와 동일한 키워드가 존재한다면
            num_words = kw['count'] # 그 키워드에 해당하는 count값이 num_words에 저장
    return num_words # 결과적으로 질의어의 출현횟수를 출력

for article in list_articles : # 모든 기사를 돌아다니며 입력받은 질의어가 몇 번 나오는지 검색
    num_query = search_articles(query, article['keywords']) # search_articles함수로 빈도수추출
    if num_query !=0 : #즉, 질의어가 하나도 없는 문서는 제외하고 출력
        print('[TF]: %2d'%num_query, end=' ')
        print('[Title]:', article['title'], end=' ')
        print('[URL]:', article['link'])

array_query = []
for article in list_articles : # 모든 기사를 돌아다니며 입력받은 질의어가 몇 번 나오는지 검색
    num_query = search_articles(query, article['keywords']) # search_articles함수로 빈도수추출
    if num_query !=0 : #즉, 질의어가 하나도 없는 문서는 제외하고 출력
        array_query.append([num_query, article['title'], article['link']])
array_query.sort(key = lambda x: -x[0]) # 내림차순정렬 "-x[0]"
print("\n** 검색 결과 **")
for index_query in array_query:
    print("[제목]", index_query[1])
    print("[URL]", index_query[2])
    print("[빈도/유사도]", index_query[0])

질의어 입력: 디지털
[TF]:  6 [Title]: '디지털 전환' 증권가, 전산운용비 늘리고 지점 줄이고 [URL]: https://www.etnews.com/20240416000158
[TF]:  8 [Title]: KISA, 오는 18일 '2024 블록체인 밋업 콘퍼런스' 개최 [URL]: https://www.etnews.com/20240416000090

** 검색 결과 **
[제목] KISA, 오는 18일 '2024 블록체인 밋업 콘퍼런스' 개최
[URL] https://www.etnews.com/20240416000090
[빈도/유사도] 8
[제목] '디지털 전환' 증권가, 전산운용비 늘리고 지점 줄이고
[URL] https://www.etnews.com/20240416000158
[빈도/유사도] 6
